In [5]:
import JupyterNotebooksLib as slicernb
import slicer
import numpy as np
import pandas as pd
import os
from tps import ThinPlateSpline

<div class="alert alert-block alert-info">
<b>In Slicer GUI</b> from the "mri/results" folder, Load the main volume of the individual (a .nrrd file) and load orientation.mrk.json; from control folder load LinearTransform.h5. Save the project in the control folder. You can choose to save a copy ot the volume in the control folder as well. The LinearTransform.h5 file has been created to transform from the OpenSim to 3DSlicer coordinates.

<p> Load all bone .stl files into the project and transform them by dragging their names onto the "LinearTransform" object in the 'Subject hierarchy' or 'Transform hierarchy' window.</p>
</div>

## Load data from Control folder

In [8]:
files = [file for file in os.listdir() if '.csv' in file]
bone_dfs = []
for file in files:
    if 'bone' in file:

        df = pd.read_csv(file, index_col='name')
        for body in ['pelvis', 'femur_l', 'femur_r', 'patella_l', 'patella_r', 'tibia_l', 'tibia_r']:
            if body in file:
                df['body'] = body
                bone_dfs.append(df)
bone_df = pd.concat(bone_dfs)
bone_df['group'] = 'bone'

muscle_dfs = []
for file in files:
    if 'muscle' in file:
        df = pd.read_csv(file, index_col='name')
        for body in ['pelvis', 'femur_l', 'femur_r', 'patella_l', 'patella_r', 'tibia_l', 'tibia_r']:
            if body in file:
                df['body'] = body
                muscle_dfs.append(df)
muscle_df= pd.concat(muscle_dfs)
muscle_df['group'] = 'muscle'

wraps_translation_dfs = []
for file in files:
    if 'wraps' in file:
        df = pd.read_csv(file, index_col='name')
        for body in ['pelvis', 'femur_l', 'femur_r', 'patella_l', 'patella_r', 'tibia_l', 'tibia_r']:
            if body in file:
                df['body'] = body
                wraps_translation_dfs.append(df)
wraps_translation_df = pd.concat(wraps_translation_dfs)
wraps_translation_df['group'] = 'wrap_transl'

In [6]:
all_df = pd.concat([bone_df, muscle_df, wraps_translation_df])
all_df.shape

(370, 5)

In [10]:
inds = {'bone':{'all':[]}, 'muscle':{'all':[]}, 'wrap':{'all':[]}}
for i in range(all_df.shape[0]):
    if all_df.iloc[i]['group'] == 'bone':
        inds['bone']['all'].append(i)
        body = all_df.iloc[i]['body']
        if body in inds['bone'].keys():
            inds['bone'][body].append(i)
        else: inds['bone'][body] = [i]
    if all_df.iloc[i]['group'] == 'muscle':
        inds['muscle']['all'].append(i)
        body = all_df.iloc[i]['body']
        if body in inds['muscle'].keys():
            inds['muscle'][body].append(i)
        else: inds['muscle'][body] = [i]
    if all_df.iloc[i]['group'] == 'wrap_transl':
        inds['wrap']['all'].append(i)
        body = all_df.iloc[i]['body']
        if body in inds['wrap'].keys():
            inds['wrap'][body].append(i)
        else: inds['wrap'][body] = [i]


In [ ]:
# load current muscle cloud, call muscles_orig, make blue
muscle_cloud = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsFiducialNode")
slicer.util.updateMarkupsControlPointsFromArray(muscle_cloud, muscle_df[['r','a','s']].to_numpy())
for i, name in enumerate(muscle_df.index):
    muscle_cloud.SetNthFiducialLabel(i, name)


In [13]:
muscle_cloud.SetName('muscles_orig')

<div class="alert alert-block alert-info">
<b>In Slicer GUI</b>
Clone muscles_orig, name the new object 'muscles_changed', change colour. You mey also want to change the size of the points. Use Murkups module->Display->Glyphsize to 2 or 1. 
</div>

## Change muscles_orig in Slicer

<div class="alert alert-block alert-info">
<b>In Slicer GUI</b>

<p> Scroll the red slice from top to bottom. As you go, check that the muscles are in the middle position. However, do not adjust all of the muscles, only the once that are clearly off the correct locations. The rest of the muscles will be 'snapped' to place with the help of the TPS function.</p>

<p>On the experience one usually needs to adjust positions of :</p> 
<li> sartor_1 and sartor_2</li>
<li> glmax3_r-P1 and glmax3_l-P1 (these may fall away from the sacrum)</li>
<li> iliacus P2 can be off center. If you change this, make sure to slightly change the position of the psoas P2 to ensure it is not changed on wrapping.
<li> addlon P1 on right and left</li>
<li> bflh P1 on right and left. It often falls inside the bone.
<li> glmax3 P3 right and left. This point typically falls onto the adductor (maybe a feature of the Rajagopal 2016 model).
<li> glmax3 P4 right and left. This point can also fall onto the adductors space.
<li> sartorius P2 right and left. This point is also often too anterior in the Rajagopal 2016 template.
<li> check that rectus and vastii points are on the outer perimeter of the patella</li>
<li> check that origins of the gastrocnemius muscles as high and as anteriorly as possible </li>
<li> check that rectus and vastii patella tendon insertions (P5s) are not outsde of the body. Changing the position of just one of these points, for example vaslat P5 should make all the rest fall into place in the next step. </li>
<li> check the origin of soleus: soleus_r-P1 and soleus_l-P1. In Rajagopal 2016 this muscle originates from the tibia. </li>
<li> tibanter_r-P3 and tibanter_l-P3</li>
<li> perlon P3 right and left </li>

</div>

In [14]:
bone_vals = np.array([pd.to_numeric(row) for row in bone_df[['r','a','s']].to_numpy()])
mscl_vals = np.array([pd.to_numeric(row) for row in muscle_df[['r','a','s']].to_numpy()])
wrp_vals = np.array([pd.to_numeric(row) for row in wraps_translation_df[['r','a','s']].to_numpy()])

In [15]:
muscles_changed = slicer.util.getNode('muscles_changed')
muscles_changed_list = slicer.util.arrayFromMarkupsControlPoints(muscles_changed)
muscles_changed_list.shape

(224, 3)

## Calculate changes and warp remaining muscles and wrapping surfaces

In [16]:
changes = []
for i, name in enumerate(muscle_df.index):
    a1 = np.around(muscles_changed_list[i], decimals=2)
    a2 = np.around(pd.to_numeric(muscle_df.loc[name, ['r','a','s']], downcast='float').to_numpy(), decimals=2)
    if np.count_nonzero(np.around(a1 - a2, decimals = 2))!=0.:
       changes.append(i)
       
res_list = [inds['muscle']['all'][i] for i in changes]
inds_for_tps = inds['bone']['all'] + res_list

changed = np.concatenate([bone_vals, muscles_changed_list[changes]])
orig = np.concatenate([bone_vals,mscl_vals[changes]])

In [17]:
len(changes)

37

In [18]:
tps_spline = ThinPlateSpline(alpha = 0.02)
tps_spline.fit(orig, changed)
# tps_spline.transform(self.osim_bone[['r','a','s']].to_numpy())
new_muscles = tps_spline.transform(mscl_vals)
new_wraps = tps_spline.transform(wrp_vals)

## Output results to Slicer

In [19]:
new_muscle_cloud = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsFiducialNode")
slicer.util.updateMarkupsControlPointsFromArray(new_muscle_cloud, new_muscles)
for i, name in enumerate(muscle_df.index):
    new_muscle_cloud.SetNthFiducialLabel(i, name)

## Record results to the control folder

<div class="alert alert-block alert-info">
<b>Before the final output</b> check that no muscle poits 'flew off' the vicinity of the body. This may happen if one of the muscle points has accidentally moved too far and the spline function was corrupted. If so, you may need to start the process again from copying 'muscles_orig'.

In [20]:
new_muscles_df = muscle_df.copy()
new_muscles_df[['r','a','s']] = new_muscles
new_muscles_df.to_csv('new_muscles.csv')

In [21]:
new_wraps_df = wraps_translation_df.copy()
new_wraps_df[['r','a','s']] = new_wraps
new_wraps_df.to_csv('new_wraps.csv')


In [22]:
new_muscles_df = pd.read_csv('new_muscles.csv', index_col= 'name')
new_wraps_df = pd.read_csv('new_wraps.csv', index_col= 'name')